Here, we will solve problems two ways
1. First using PySpark function 
2. Second using Spark SQL

In [1]:
# First Load all the required library and also Start Spark Session
# Load all the required library
from pyspark.sql import SparkSession

In [2]:
#Start Spark Session
spark = SparkSession.builder.appName("problem2").getOrCreate()
sqlContext = SparkSession(spark)
#Dont Show warning only error
spark.sparkContext.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/08 11:06:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#Load CSV file into DataFrame
employeedf = spark.read.format("csv").option("header","true").option("inferSchema","true").load("employee_salary.csv")

In [4]:
#Check Schema of DataFrame
employeedf.printSchema()

root
 |-- id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- department_id: integer (nullable = true)



In [5]:
#Check sample Data 
employeedf.show()

+---+----------+---------+------+-------------+
| id|first_name|last_name|salary|department_id|
+---+----------+---------+------+-------------+
| 45|     Kevin|   Duncan| 45210|         1003|
| 25|    Pamela| Matthews| 57944|         1005|
| 48|    Robert|    Lynch|117960|         1004|
| 34|    Justin|     Dunn| 67992|         1003|
| 62|      Dale|    Hayes| 97662|         1005|
|  1|      Todd|   Wilson|110000|         1006|
| 61|      Ryan|    Brown|120000|         1003|
| 21|   Stephen|    Berry|123617|         1002|
| 13|     Julie|  Sanchez|210000|         1001|
| 55|   Michael|   Morris|106799|         1005|
| 44|    Trevor|   Carter| 38670|         1001|
| 73|   William|  Preston|155225|         1003|
| 39|     Linda|    Clark|186781|         1002|
| 10|      Sean| Crawford|190000|         1006|
| 30|   Stephen|    Smith|194791|         1001|
| 75|     Julia|    Ramos|105000|         1006|
| 59|     Kevin| Robinson|100924|         1005|
| 69|    Ernest| Peterson|115993|       

In [6]:
#Load CSV file into DataFrame
departmentdf = spark.read.format("csv").option("header","true").option("inferSchema","true").load("department.csv")

In [8]:
#Check Schema of DataFrame
departmentdf.printSchema()

root
 |-- department_id: integer (nullable = true)
 |-- department_name: string (nullable = true)



In [9]:
#Check sample Data 
departmentdf.show()

+-------------+---------------+
|department_id|department_name|
+-------------+---------------+
|         1005|          Sales|
|         1002|       Finanace|
|         1004|       Purchase|
|         1001|     Operations|
|         1006|      Marketing|
|         1003|      Technoogy|
+-------------+---------------+



In [16]:
#Solving Problem using PySpark 
# 1. Use this both tables and list all the employees woking in marketing department with highest to lowest salary order. 

joineddf = employeedf.join(departmentdf, employeedf.department_id == departmentdf.department_id,"left")
joineddf.show()

+---+----------+---------+------+-------------+-------------+---------------+
| id|first_name|last_name|salary|department_id|department_id|department_name|
+---+----------+---------+------+-------------+-------------+---------------+
| 45|     Kevin|   Duncan| 45210|         1003|         1003|      Technoogy|
| 25|    Pamela| Matthews| 57944|         1005|         1005|          Sales|
| 48|    Robert|    Lynch|117960|         1004|         1004|       Purchase|
| 34|    Justin|     Dunn| 67992|         1003|         1003|      Technoogy|
| 62|      Dale|    Hayes| 97662|         1005|         1005|          Sales|
|  1|      Todd|   Wilson|110000|         1006|         1006|      Marketing|
| 61|      Ryan|    Brown|120000|         1003|         1003|      Technoogy|
| 21|   Stephen|    Berry|123617|         1002|         1002|       Finanace|
| 13|     Julie|  Sanchez|210000|         1001|         1001|     Operations|
| 55|   Michael|   Morris|106799|         1005|         1005|   

In [20]:
from pyspark.sql.functions import desc
joineddf.select("first_name","last_name","salary").where("department_name='Marketing'").orderBy(desc("salary")).show()

+----------+---------+------+
|first_name|last_name|salary|
+----------+---------+------+
|      Sean| Crawford|190000|
|  Danielle| Williams|120000|
|      Todd|   Wilson|110000|
|     Julia|    Ramos|105000|
|      Eric|Zimmerman| 83093|
|     Jason|    Olsen| 51937|
|     Jason|  Burnett| 42525|
|    Philip|Gillespie| 36424|
+----------+---------+------+



In [21]:
# Now we are solving Same problem using Spark SQL 
# Creating Temp Table or HIVE table
employeedf.createOrReplaceTempView("tmpEmployee")
departmentdf.createOrReplaceTempView("tmpDepartment")

In [22]:
# Now we have SQL Table and we can write SQL Query on top of that 
# For example by Select on table 
sqlContext.sql("SELECT * FROM tmpEmployee").show()

+---+----------+---------+------+-------------+
| id|first_name|last_name|salary|department_id|
+---+----------+---------+------+-------------+
| 45|     Kevin|   Duncan| 45210|         1003|
| 25|    Pamela| Matthews| 57944|         1005|
| 48|    Robert|    Lynch|117960|         1004|
| 34|    Justin|     Dunn| 67992|         1003|
| 62|      Dale|    Hayes| 97662|         1005|
|  1|      Todd|   Wilson|110000|         1006|
| 61|      Ryan|    Brown|120000|         1003|
| 21|   Stephen|    Berry|123617|         1002|
| 13|     Julie|  Sanchez|210000|         1001|
| 55|   Michael|   Morris|106799|         1005|
| 44|    Trevor|   Carter| 38670|         1001|
| 73|   William|  Preston|155225|         1003|
| 39|     Linda|    Clark|186781|         1002|
| 10|      Sean| Crawford|190000|         1006|
| 30|   Stephen|    Smith|194791|         1001|
| 75|     Julia|    Ramos|105000|         1006|
| 59|     Kevin| Robinson|100924|         1005|
| 69|    Ernest| Peterson|115993|       

In [23]:
sqlContext.sql("SELECT * FROM tmpDepartment").show()

+-------------+---------------+
|department_id|department_name|
+-------------+---------------+
|         1005|          Sales|
|         1002|       Finanace|
|         1004|       Purchase|
|         1001|     Operations|
|         1006|      Marketing|
|         1003|      Technoogy|
+-------------+---------------+



In [24]:
# Now we will write query to get max salary for each employee 
# so we will use SQL Group by and SQL Order by functions 
sqlContext.sql("SELECT first_name, last_name, salary \
                       FROM tmpEmployee as emp \
                       LEFT OUTER JOIN tmpDepartment as department \
                       ON emp.department_id = department.department_id \
                       WHERE department.department_name = 'Marketing' \
                       ORDER BY salary DESC").show(n=100)



+----------+---------+------+
|first_name|last_name|salary|
+----------+---------+------+
|      Sean| Crawford|190000|
|  Danielle| Williams|120000|
|      Todd|   Wilson|110000|
|     Julia|    Ramos|105000|
|      Eric|Zimmerman| 83093|
|     Jason|    Olsen| 51937|
|     Jason|  Burnett| 42525|
|    Philip|Gillespie| 36424|
+----------+---------+------+

